In [2]:
import os
import numpy as np
import torch
from PIL import Image

from torch import nn, optim

import visionutils.transforms as T
from visionutils.engine import train_one_epoch, evaluate
import visionutils.utils

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torchvision.models import resnet18, ResNet18_Weights

import matplotlib.pyplot as plt

from utils.nn import ClusterDataset, get_transform, get_instance_segmentation_model, get_instance_frcnn_model
from utils.evaluate import make_cat, reduce_cat, eval_model

%load_ext autoreload
%autoreload 2

%matplotlib inline

ImportError: attempted relative import with no known parent package

In [ ]:
#This function computes the inverse-variance weighted average of a data vector
def inv_var(data, variances):
    ave = 0
    var = 0
    for i in range(len(data)):
        ave += data[i]/variances[i]**2
        var += 1/variances[i]**2
    return ave/var, np.sqrt(1/var)

#This function evaluates the performance of a model on a single image
def eval_img(img, truth, model, threshold = 0.5):
    xs = []
    ys = []
    errs = []

    # put the model in evaluation mode
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])
    for i in range(len(prediction[0]['boxes'])):
        if prediction[0]['scores'][i]>=threshold:
            xmin, ymin, xmax, ymax = prediction[0]['boxes'][i].cpu().numpy()
            xs.append((xmax+xmin)/2)
            ys.append((ymax+ymin)/2)
            errs.append(prediction[0]['scores'][i].cpu().numpy())
            #print(xs[-1], ys[-1])
    
    print("X loc: ", inv_var(xs, errs))
    print("Y loc: ", inv_var(ys, errs))

# Begin Training

In [ ]:
tile_type = "indv"

In [ ]:

#root = '/project/r/rbond/jorlo/datasets/ACT_tiles/'
root = "/mnt/welch/USERS/jorlo/ml-clusters/ACT_tiles"
len(list(sorted(os.listdir(os.path.join(root, "{}_masks".format(tile_type))))))

In [ ]:
# use our dataset and defined transformations
dataset = ClusterDataset(root, get_transform(train=True),
                        cluster_dir = '{}_freq_stamps'.format(tile_type),
                        mask_dir = '{}_freq_masks'.format(tile_type))
dataset_test = ClusterDataset(root, get_transform(train=False),
                        cluster_dir = '{}_freq_stamps'.format(tile_type), 
                        mask_dir = '{}_freq_masks'.format(tile_type))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()

test_num = 20
dataset = torch.utils.data.Subset(dataset, indices[:-test_num])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-test_num:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=8, shuffle=True, num_workers=4,
    collate_fn=visionutils.utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=visionutils.utils.collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

backbone = 'mobilenet'
backbone_path = "/mnt/welch/USERS/jorlo/ml-clusters/models/torch-act/"
# get the model using our helper function
model = get_instance_frcnn_model(num_classes, backbone_path = backbone_path + "act-{}.pth".format(backbone), backbone_type = backbone)
#model = get_instance_frcnn_model(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
# let's train it for 10 epochs
from torch.optim.lr_scheduler import StepLR
num_epochs = 10

model_path = "/mnt/welch/USERS/jorlo/ml-clusters/models/torch-act/act-{}-frcnn-{}-tiles.pth".format(backbone, tile_type)
load_exiting_weights = True
if load_exiting_weights and os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)
    torch.save(model.state_dict(), model_path)


### Reload model

In [ ]:
#model.load_state_dict(torch.load("/project/r/rbond/jorlo/ml-clusters/models/torch-act/act-mobilenet-frcnn.pth"))
#model.load_state_dict(torch.load(model_path + "act-{}-frcnn-indv.pth".format(backbone)))
#model_path = "/mnt/welch/USERS/jorlo/ml-clusters/models/torch-act/"

#model.load_state_dict(torch.load(model_path+"act-{}-frcnn-{}-tiles.pth".format(backbone, tile_type)))
#model.load_state_dict(torch.load("/mnt/welch/USERS/jorlo/ml-clusters/models/mist-torch-act/act-resnet-frcnn-tiles.pth"))

In [ ]:
prediction

In [ ]:
plt.imshow(img[1,...])
plt.show()

In [ ]:
for j in range(10):

    img, truth = dataset_test[j]
    # put the model in evaluation mode
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])
    print(truth)
    print(prediction)
    print('\n')

In [ ]:
img_path = "/mnt/welch/USERS/jorlo/ml-clusters/ACT_tiles/indv_freq_stamps/{:04}.fits"

cents = make_catalog(model, dataset_test, img_path)

In [ ]:
cat = reduce_cat(cents)

In [ ]:
cat

In [ ]:
cents

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.imshow(img[0,...])

colors = ["red", "black", "purple", "green", "yellow"]

for i in range(len(prediction[0]['boxes'])):
    if prediction[0]['scores'][i]>=0.5:
        xmin, ymin, xmax, ymax = prediction[0]['boxes'][i].cpu().numpy()
        rectangle = plt.Rectangle((xmin,ymin), xmax-xmin,ymax-ymin, fc='none',ec=colors[i], label = 'Guess')
        ax.add_patch(rectangle)
        
for i in range(len(truth['boxes'])):
    xmin, ymin, xmax, ymax = truth['boxes'][i].cpu().numpy()
    rectangle = plt.Rectangle((xmin,ymin), xmax-xmin,ymax-ymin, fc='none',ec=colors[i], linestyle = '--', label = 'Truth')
    ax.add_patch(rectangle)

plt.legend()
plt.savefig('./figs/region_prop_{}.png'.format(j))
plt.savefig('./figs/region_prop_{}.pdf'.format(j))

plt.show()
plt.close()

In [ ]:
len(prediction[0]['boxes'])

In [ ]:
for j in range(10):
    img, truth = dataset_test[j]
    # put the model in evaluation mode
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.imshow(img[0,...])

    colors = ["red", "black", "purple", "green", "yellow"]

    for i in range(len(prediction[0]['boxes'])):
        if prediction[0]['scores'][i]>=0.5:
            xmin, ymin, xmax, ymax = prediction[0]['boxes'][i].cpu().numpy()
            rectangle = plt.Rectangle((xmin,ymin), xmax-xmin,ymax-ymin, fc='none',ec=colors[i], label = 'Guess')
            ax.add_patch(rectangle)

    for i in range(len(truth['boxes'])):
        xmin, ymin, xmax, ymax = truth['boxes'][i].cpu().numpy()
        rectangle = plt.Rectangle((xmin,ymin), xmax-xmin,ymax-ymin, fc='none',ec=colors[i], linestyle = '--', label = 'Truth')
        ax.add_patch(rectangle)

    plt.legend()
    plt.savefig('./figs/region_prop_{}.png'.format(j))
    plt.savefig('./figs/region_prop_{}.pdf'.format(j))

    plt.show()
    plt.close()

# Garbage

In [ ]:
for j in range(455):
    mask_list = list(sorted(os.listdir(os.path.join('/project/r/rbond/jorlo/datasets/ACT_tiles/', "masks"))))
    mask_path = os.path.join('/project/r/rbond/jorlo/datasets/ACT_tiles/', "masks", mask_list[j])
    with np.load(mask_path) as data:
        mask = data['arr_0']
    # convert the PIL Image into a numpy array
    mask = np.array(mask)
    # instances are encoded as different colors
    obj_ids = np.unique(mask)
    # first id is the background, so remove it
    obj_ids = obj_ids[1:]
    # split the color-encoded mask into a set
    # of binary masks
    masks = mask == obj_ids[:, None, None]
    # get bounding box coordinates for each mask
    num_objs = len(obj_ids)
    boxes = []
    for i in range(num_objs):
        pos = np.where(masks[i])
        xmin = np.min(pos[1])
        xmax = np.max(pos[1])
        ymin = np.min(pos[0])
        ymax = np.max(pos[0])
        boxes.append([xmin, ymin, xmax, ymax])
        if xmin==xmax and ymin == ymax:
            print(mask_list[j])
        
        

    # convert everything into a torch.Tensor
    boxes = torch.as_tensor(boxes, dtype=torch.float32)

In [ ]:
path = root + "/"
tile = '1_1_0'


with np.load(path+'small_freq_tiles/'+tile+'.npz') as data:
        img = data['arr_0']
with np.load(path+'small_masks/'+tile+'_mask.npz') as data:
        mask = data['arr_0']
img = np.array(img)
mask = np.array(mask)
# instances are encoded as different colors
obj_ids = np.unique(mask)
# first id is the background, so remove it
obj_ids = obj_ids[1:]
# split the color-encoded mask into a set
# of binary masks
masks = mask == obj_ids[:, None, None]
# get bounding box coordinates for each mask
num_objs = len(obj_ids)
boxes = []
for i in range(num_objs):
    pos = np.where(masks[i])
    xmin = np.min(pos[1])
    xmax = np.max(pos[1])
    ymin = np.min(pos[0])
    ymax = np.max(pos[0])
    boxes.append([xmin, ymin, xmax, ymax])

img = np.ascontiguousarray(img.transpose(2,0,1))
model.eval()

img = torch.as_tensor(img, dtype=torch.float32)
with torch.no_grad():
    prediction = model([img.to(device)])